In [1]:
import pandas as pd
import numpy as np
from src.data_loader import *
from src.strategy import *
from src.index import *
from src.ticker_selection import *

In [2]:
data = DataGenerator('db').data_search(all=True)

기존 데이터 불러오기 성공


In [5]:
def add_feature(df, **kwargs):
    data = df.copy()
    data["rsi"] = rsi(data)
    data[["macd","macd_signal","macd_oscillator"]] = macd(data, **kwargs)
    data[["en_center","en_ub","en_lb"]]            = envelope(data, **kwargs)
    data[["bo_center","bo_ub","bo_lb"]]            = bollinger(data, **kwargs)
    data[["slow_k","slow_d"]]                      = stochastic(data, **kwargs)
    data["sma(5)"]                                 = sma(data, 5)
    data["sma(20)"]                                = sma(data, 20) 
    data["sma(60)"]                                = sma(data, 60) 
    data["sma(120)"]                               = sma(data, 120)
    # data["sar"]                                    = sar(data, **kwargs)
    return data

In [8]:
data_list = []
for TICKER in tqdm(data['Ticker'].unique()):
    _ = data[data['Ticker']== TICKER]
    _['r_signal'] = rsi_strategy(_, 70, 30)
    _['macd_signal'] = macd_strategy(_)
    _['envelope_signal'] = envelope_strategy(_)
    _['bollinger_signal'] = bollinger_strategy(_)
    _['stochastic_signal'] = stochastic_strategy(_)
    data_list.append(_)
data_concat = pd.concat(data_list, axis = 0)

100%|██████████| 4439/4439 [16:21:50<00:00, 13.27s/it]    


In [11]:
data_concat.to_csv('./concat.csv')

In [13]:
pd.read_csv('./concat.csv' ,index_col=0)

,Datetime,Open,High,Low,Close,Adj Close,Volume,Ticker,r_signal,macd_signal,envelope_signal,bollinger_signal,stochastic_signal
0,2008-01-29,9.50,9.99,8.57,8.75,0.702589,1489000.0,AACG,0,0,0,0,0
1,2008-01-30,8.75,9.15,8.30,8.50,0.682516,219000.0,AACG,0,0,0,0,0
2,2008-01-31,8.49,10.30,8.49,9.55,0.766826,182300.0,AACG,0,0,0,0,0
3,2008-02-01,9.93,9.94,9.50,9.51,0.763614,28200.0,AACG,0,0,0,0,0
4,2008-02-04,9.50,9.71,9.50,9.50,0.762811,8300.0,AACG,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
11044000,2022-10-28,9.73,10.90,9.61,10.52,10.520000,696000.0,ZYXI,0,1,-1,-1,-1
11044001,2022-10-31,10.40,11.50,10.35,11.40,11.400000,441700.0,ZYXI,-1,1,-1,-1,-1
11044002,2022-11-01,11.90,12.44,11.68,12.27,12.270000,537300.0,ZYXI,-1,1,-1,-1,-1
11044003,2022-11-02,12.31,12.91,11.99,12.56,12.560000,403900.0,ZYXI,-1,1,-1,-1,-1
